In [5]:
import numpy as np, pandas as pd, os
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model

WIN, HORIZON = 24, 24

def load_series():
    if os.path.exists("d:/data/chart/data.csv") :
        df = pd.read_csv("d:/data/chart/data.csv")
        if "ts" in df.columns:
            df["ts"] = pd.to_datetime(df["ts"])
            df = df.sort_values("ts")
            return pd.Series(df["y"].values, index=df["ts"])
        else:
            return pd.Series(df.iloc[:, 0].values)
    else:
        idx = pd.date_range("2024-01-01", periods=60*24, freq="H")
        y = 0.6 + 0.4*np.sin(2*np.pi*(idx.hour/24.0)) + np.random.normal(0, 0.05, size=len(idx))
        y = np.clip(y, 0.1, None)
        return pd.Series(y, index=idx)

def make_xy(series: pd.Series, win=WIN, horizon=HORIZON):
    arr = series.values.astype("float32")
    X, Y = [], []
    for i in range(len(arr)-win-horizon):
        X.append(arr[i:i+win])
        Y.append(arr[i+win:i+win+horizon])
    X = np.array(X)[...,None]
    Y = np.array(Y)
    return X, Y

series = load_series()
X, Y = make_xy(series)

model = Sequential([LSTM(32, input_shape=(WIN, 1)), Dense(HORIZON)])
model.compile(optimizer="adam", loss="mean_squared_error")

es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X, Y, epochs=30, batch_size=64, validation_split=0.1, callbacks=[es], verbose=2)

save_model(model, "d:/data/chart/model_lstm.h5")
print("Saved model_lstm.h5")

Epoch 1/30


c:\python3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 - 2s - 133ms/step - loss: 1.1211 - val_loss: 0.7527
Epoch 2/30
14/14 - 0s - 13ms/step - loss: 0.7446 - val_loss: 0.3525
Epoch 3/30
14/14 - 0s - 13ms/step - loss: 0.3114 - val_loss: 0.1420
Epoch 4/30
14/14 - 0s - 12ms/step - loss: 0.1397 - val_loss: 0.0620
Epoch 5/30
14/14 - 0s - 13ms/step - loss: 0.0952 - val_loss: 0.0530
Epoch 6/30
14/14 - 0s - 12ms/step - loss: 0.0907 - val_loss: 0.0525
Epoch 7/30
14/14 - 0s - 12ms/step - loss: 0.0894 - val_loss: 0.0509
Epoch 8/30
14/14 - 0s - 12ms/step - loss: 0.0886 - val_loss: 0.0514
Epoch 9/30
14/14 - 0s - 12ms/step - loss: 0.0878 - val_loss: 0.0507
Epoch 10/30
14/14 - 0s - 13ms/step - loss: 0.0872 - val_loss: 0.0497
Epoch 11/30
14/14 - 0s - 11ms/step - loss: 0.0863 - val_loss: 0.0504
Epoch 12/30
14/14 - 0s - 12ms/step - loss: 0.0850 - val_loss: 0.0484
Epoch 13/30
14/14 - 0s - 11ms/step - loss: 0.0833 - val_loss: 0.0478
Epoch 14/30
14/14 - 0s - 11ms/step - loss: 0.0806 - val_loss: 0.0461
Epoch 15/30
14/14 - 0s - 12ms/step - loss: 0.0765 - v

Saved model_lstm.h5
